In [ ]:
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Load FAISS index & metadata
index_file = "faiss.index"
metadata_file = "metadata.pkl"

index = faiss.read_index(index_file)
with open(metadata_file, "rb") as f:
    metadata = pickle.load(f)

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Load Gemma LLM
gemma = pipeline("text-generation", model="google/gemma-2b")

def filter_metadata(filter_criteria):
    """Filters metadata and returns a list of valid indices."""
    filtered_indices = []
    for entry in metadata:
        if all(entry[key] == value for key, value in filter_criteria.items()):
            filtered_indices.append(entry["id"])
    return filtered_indices

def search_plans(query, filter_criteria={}):
    """Search FAISS with metadata filtering."""
    query_embedding = model.encode(query).astype("float32").reshape(1, -1)

    # Apply metadata filtering
    if filter_criteria:
        valid_indices = filter_metadata(filter_criteria)
        if not valid_indices:
            return "No matching plans found."
        
        # Use FAISS to search only within the filtered indices
        index_subset = faiss.IndexFlatL2(index.d)
        index_subset.add(index.reconstruct_n(valid_indices, len(valid_indices)))

        _, indices = index_subset.search(query_embedding, k=3)
        indices = [valid_indices[i] for i in indices[0] if i < len(valid_indices)]
    else:
        _, indices = index.search(query_embedding, k=3)
        indices = [i for i in indices[0] if i < len(metadata)]

    if not indices:
        return "No relevant plans found."

    # Prepare response
    response = "\n".join([
        f"{metadata[i]['Plan Name']} by {metadata[i]['Provider Name']} - {metadata[i]['Monthly Price']}" 
        for i in indices
    ])

    return response

# Example Query
query = "What is the best unlimited data plan under $70?"
filters = {"Payment Frequency": "1 Month"}  # Example filter
response = search_plans(query, filters)
print("🔍 Relevant Plans:\n", response)

# Use Gemma LLM to generate a response
llm_input = f"User is looking for: {query}\nRelevant plans:\n{response}\nProvide a helpful response."
llm_output = gemma(llm_input, max_length=200)
print("\n🤖 AI Response:\n", llm_output[0]["generated_text"])
